## Hands on Iceberg with AWS Athena

Before you go through this notebook, please check [README](./README.md) for a complete guide on how to set up everything.

This notebook tries to help you to try AWS Athena on a easy a reproducible way.
To start using this notebook, please select your `AWS_PROFILE` on the cell below.

In [1]:
# Initial Run
%env AWS_PROFILE=given_profile_name

import os

from dotenv import load_dotenv

from magic_cells import get_var

if not load_dotenv(f"{os.getcwd()}/terraform.env"):
    raise ImportError("terraform.env not does not exist. You might need to run terraform first.")

DATA_BUCKET = os.environ["DATA_BUCKET"]
print(f"{DATA_BUCKET = }")


env: AWS_PROFILE=illo
DATA_BUCKET = 'illo-laz-iceberg-data'


In [2]:
# Only run this cell once to upload the CSV to the S3 bucket
import requests
from io import BytesIO
from zipfile import ZipFile

import boto3

DATASET_URL = "https://github.com/awesomedata/awesome-public-datasets/raw/master/Datasets/titanic.csv.zip"
CSV_FOLDER = "csv/"

# Download the dataset and open the zip file
response_bytes = BytesIO(requests.get(DATASET_URL).content)
zip_file = ZipFile(response_bytes)

# Upload CSV file to S3 Bucket
s3 = boto3.client("s3")
print(f"\nUploading files to '{DATA_BUCKET}/{CSV_FOLDER}'")

for file in zip_file.namelist():
    if not file.startswith("__MACOSX") and file.endswith(".csv"):
        csv_file = zip_file.open(file)
        s3.upload_fileobj(csv_file, DATA_BUCKET, CSV_FOLDER + file)
        print(f"    file '{file}' uploaded successfully.")


Uploading files to 'illo-laz-iceberg-data/csv/'
    file 'titanic.csv' uploaded successfully.


### Notebook with magic

I created some magic cell commands that will make this notebook more enjoyable.

- athena_exec: Its intended to be a function to edit your table or table schema. You could either create a table, delete it, populate or any action that requires any write. Its usage is very simple: `%%athena_exec <your_database>`

- athena: Allows you to read data from Athena using SQL into a python dataframe.<br/>
The usage will be `%%athena <your_database> <(Optional) python_variable_name>`

Find examples of usage in the three following cells.

In [3]:
%%athena_exec iceberg_catalog
-- This query will create a Glue table pointing to our CSVs files
CREATE EXTERNAL TABLE passenger_data_csv
(
    PassengerId bigint,
    Survived bigint,
    Pclass bigint,
    Name string,
    Sex string,
    Age double,
    SibSp bigint,
    Parch bigint,
    Ticket string,
    Fare double,
    Cabin string,
    Embarked string
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 's3://{DATA_BUCKET}/csv/';


Executing SQL on Database: iceberg_catalog


{'QueryExecutionId': '8f353afa-a7fd-4c53-a647-938abc7963f4',
 'ResponseMetadata': {'RequestId': '64b144da-9b85-4b8b-8f5b-60d15cb563d0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Jun 2024 13:14:31 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '64b144da-9b85-4b8b-8f5b-60d15cb563d0'},
  'RetryAttempts': 0}}

In [4]:
%%athena iceberg_catalog df
-- Simple select that saves the result as Pandas dataframe in the variable you defined
SELECT * FROM passenger_data_csv LIMIT 10

Executing SQL on Database: iceberg_catalog
SQL Query to execute
-- Simple select that saves the result as Pandas dataframe in the variable you defined
SELECT * FROM passenger_data_csv LIMIT 10

Saving result of the SQL in variable: df


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,<NA>,<NA>,<NA>,Name,Sex,NaN,<NA>,<NA>,Ticket,NaN,Cabin,Embarked
1,1,0,3,"""Braund","Mr. Owen Harris""",NaN,22,1,0,NaN,7.25,
2,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",NaN,38,1,0,NaN,71.2833,C85
3,3,1,3,"""Heikkinen","Miss. Laina""",NaN,26,0,0,NaN,7.925,
4,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",NaN,35,1,0,113803.0,53.1,C123
5,5,0,3,"""Allen","Mr. William Henry""",NaN,35,0,0,373450.0,8.05,
6,6,0,3,"""Moran","Mr. James""",NaN,<NA>,0,0,330877.0,8.4583,
7,7,0,1,"""McCarthy","Mr. Timothy J""",NaN,54,0,0,17463.0,51.8625,E46
8,8,0,3,"""Palsson","Master. Gosta Leonard""",NaN,2,3,1,349909.0,21.075,
9,9,1,3,"""Johnson","Mrs. Oscar W (Elisabeth Vilhelmina Berg)""",NaN,27,0,2,347742.0,11.1333,


In [5]:
df = get_var('df')
df

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,<NA>,<NA>,<NA>,Name,Sex,NaN,<NA>,<NA>,Ticket,NaN,Cabin,Embarked
1,1,0,3,"""Braund","Mr. Owen Harris""",NaN,22,1,0,NaN,7.25,
2,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",NaN,38,1,0,NaN,71.2833,C85
3,3,1,3,"""Heikkinen","Miss. Laina""",NaN,26,0,0,NaN,7.925,
4,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",NaN,35,1,0,113803.0,53.1,C123
5,5,0,3,"""Allen","Mr. William Henry""",NaN,35,0,0,373450.0,8.05,
6,6,0,3,"""Moran","Mr. James""",NaN,<NA>,0,0,330877.0,8.4583,
7,7,0,1,"""McCarthy","Mr. Timothy J""",NaN,54,0,0,17463.0,51.8625,E46
8,8,0,3,"""Palsson","Master. Gosta Leonard""",NaN,2,3,1,349909.0,21.075,
9,9,1,3,"""Johnson","Mrs. Oscar W (Elisabeth Vilhelmina Berg)""",NaN,27,0,2,347742.0,11.1333,


## Working with Iceberg

Here you have two queries. On the first one, you define the table, where to store the data and metadata and the structure of it.
The second one, uses the previous data from a CSV file to insert those values in there.
You can go to your s3 bucket and you'll see all the files being created there following the Iceberg specs.


In [6]:
%%athena_exec iceberg_catalog
-- This query will create a Glue table for Iceberg
CREATE TABLE passenger_data_iceberg
(
    PassengerId bigint,
    Survived bigint,
    Pclass bigint,
    Name string,
    Sex string,
    Age double,
    SibSp bigint,
    Parch bigint,
    Ticket string,
    Fare double,
    Cabin string,
    Embarked string
)
LOCATION 's3://{DATA_BUCKET}/passenger_iceberg/'
TBLPROPERTIES ( 'table_type' ='ICEBERG'  );


Executing SQL on Database: iceberg_catalog


{'QueryExecutionId': 'fdf29987-7929-4a9f-b874-70f02bb52366',
 'ResponseMetadata': {'RequestId': 'c038a466-9284-4db6-9603-bb6ebe4327fc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Jun 2024 13:14:52 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c038a466-9284-4db6-9603-bb6ebe4327fc'},
  'RetryAttempts': 0}}

In [7]:
%%athena_exec iceberg_catalog
-- Load data into the iceberg table from csv table
INSERT INTO passenger_data_iceberg
SELECT * FROM passenger_data_csv ;

Executing SQL on Database: iceberg_catalog


{'QueryExecutionId': 'd0acc07d-78ca-42b9-b3db-0f32bee80b6d',
 'ResponseMetadata': {'RequestId': 'd3f34fee-5c12-4b23-9703-294d33e1d625',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Jun 2024 13:15:01 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd3f34fee-5c12-4b23-9703-294d33e1d625'},
  'RetryAttempts': 0}}

In [8]:
%%athena iceberg_catalog df
-- Simple select that saves the result as Pandas dataframe in the variable you defined
SELECT * FROM passenger_data_iceberg LIMIT 10

Executing SQL on Database: iceberg_catalog
SQL Query to execute
-- Simple select that saves the result as Pandas dataframe in the variable you defined
SELECT * FROM passenger_data_iceberg LIMIT 10

Saving result of the SQL in variable: df


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,<NA>,<NA>,<NA>,Name,Sex,NaN,<NA>,<NA>,Ticket,NaN,Cabin,Embarked
1,1,0,3,"""Braund","Mr. Owen Harris""",NaN,22,1,0,NaN,7.25,
2,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",NaN,38,1,0,NaN,71.2833,C85
3,3,1,3,"""Heikkinen","Miss. Laina""",NaN,26,0,0,NaN,7.925,
4,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",NaN,35,1,0,113803.0,53.1,C123
5,5,0,3,"""Allen","Mr. William Henry""",NaN,35,0,0,373450.0,8.05,
6,6,0,3,"""Moran","Mr. James""",NaN,<NA>,0,0,330877.0,8.4583,
7,7,0,1,"""McCarthy","Mr. Timothy J""",NaN,54,0,0,17463.0,51.8625,E46
8,8,0,3,"""Palsson","Master. Gosta Leonard""",NaN,2,3,1,349909.0,21.075,
9,9,1,3,"""Johnson","Mrs. Oscar W (Elisabeth Vilhelmina Berg)""",NaN,27,0,2,347742.0,11.1333,
